In [5]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

In [6]:
%aimport caplib
from caplib import show, break_captcha, load_random_img
import caplib
import cv2

In [7]:
from IPython.display import clear_output

In [8]:
img_count = 90
for i in range(img_count+1):
    clear_output()
    print(i,"/",img_count)
    (fname,img) = load_random_img(i)
    answ = break_captcha(img)
    dirmane = "/home/vdimir/userdata2/cvis/captcha/_out/"
    cv2.imwrite(dirmane + answ + ".jpg", img)
# "img/4/%d.jpg" % fname
# show(img)

5 / 90


KeyboardInterrupt: 